In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline 
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
from torch.autograd import Variable

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
    
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.lstm_1 = nn.LSTM(self.input_size, self.hidden_size)
        self.lstm_2 = nn.LSTM(self.hidden_size, self.hidden_size)
        self.lstm_3 = nn.LSTM(self.hidden_size, self.hidden_size)
        self.lstm_4 = nn.LSTM(self.hidden_size, self.hidden_size)
        self.lstm_5 = nn.LSTM(self.hidden_size, self.hidden_size)
        self.lstm_6 = nn.LSTM(self.hidden_size, self.hidden_size)
        self.lstm_7 = nn.LSTM(self.hidden_size, self.hidden_size)
        
        self.out = nn.Linear(self.hidden_size, self.output_size)
          
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input, ch_1, ch_2, ch_3, ch_4, ch_5, ch_6, ch_7):
        (hidden_1, cell_1), (hidden_2, cell_2), (hidden_3, cell_3), (hidden_4, cell_4), (hidden_5, cell_5) = ch_1, ch_2, ch_3, ch_4, ch_5
        (hidden_6, cell_6), (hidden_7, cell_7) = ch_6, ch_7
        
        
        output, (hidden_1, cell_1) = self.lstm_1(input.view(1,1,-1).float(), (hidden_1, cell_1))
        output_1 = output
        
        output, (hidden_2, cell_2) = self.lstm_2(output, (hidden_2, cell_2))
        output_2 = output
        
        output, (hidden_3, cell_3) = self.lstm_3(output + output_1, (hidden_3, cell_3)) # skip_connection 1
        output_3 = output
        
        output, (hidden_4, cell_4) = self.lstm_4(output + output_2, (hidden_4, cell_4)) # skip_connection 2
        output_4 = output
        
        output, (hidden_5, cell_5) = self.lstm_5(output + output_3, (hidden_5, cell_5)) # skip_connection 3
        output_5 = output
        
        output, (hidden_6, cell_6) = self.lstm_6(output + output_4, (hidden_6, cell_6)) # skip_connection 4
        
        output, (hidden_7, cell_7) = self.lstm_7(output + output_5, (hidden_7, cell_7)) # skip_connection 5
        
        output = self.out(output[0])
        output = self.softmax(output)
        return output, (hidden_1, cell_1),(hidden_2, cell_2),(hidden_3, cell_3),(hidden_4, cell_4),(hidden_5, cell_5),(hidden_6, cell_6),(hidden_7, cell_7)
    
    def init_hidden(self):
        return torch.rand((1, 1, self.hidden_size), device=device)/100
    
    def init_cell(self):
        return torch.rand((1, 1, self.hidden_size), device=device)/100
    
    
print(torch.cuda.is_available())



""" 
NOTE: 
Encoder RNN input of size (Sentence_length * input_feature)
Encoder RNN output of size (1 * 1 * hidden_size)  should be (num_sentences * 1 * hidden_size)

Decoder RNN input of size 0 (scalar value)
Decoder RNN output of size (1 * target_num)   should be (num_sentences * target_num)
"""

True


' \nNOTE: \nEncoder RNN input of size (Sentence_length * input_feature)\nEncoder RNN output of size (1 * 1 * hidden_size)  should be (num_sentences * 1 * hidden_size)\n\nDecoder RNN input of size 0 (scalar value)\nDecoder RNN output of size (1 * target_num)   should be (num_sentences * target_num)\n'

In [7]:
import pickle

# load data from file
"""with open("/home/yiqin/2018summer_project/DeepMusic/pitch_data.pkl", "rb") as f:
    dic = pickle.load(f)
    train_X = dic["X"]
    #train_Y = dic["Y"]
    #time_X = dic["time"]
    """
    
with open("/home/yiqin/2018summer_project/balanced_data_small.pkl", "rb") as f:
    dic = pickle.load(f)
    train_X = dic["X2"]
    train_Y = dic["Y"]
    
    
target_Tensor = train_Y
maximum_target = len(train_Y)

In [8]:
def focal_loss(gamma, rescale, criterion, output, target):
    if int(target) == 1:
        p_negative = (1 - output[0,1])**gamma
        loss = rescale * p_negative * criterion(output, target.unsqueeze(0).long())
    else:
        p_negative = (1 - output[0,0])**gamma
        loss = p_negative * criterion(output, target.unsqueeze(0).long())
    return loss

In [9]:
import random
teacher_forcing_ratio = 1


def train(input_tensor, target_tensor, decoder, decoder_optimizer, criterion, verbose = False):
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    hidden_1 = decoder.init_hidden()
    hidden_2 = decoder.init_hidden()
    hidden_3 = decoder.init_hidden()
    hidden_4 = decoder.init_hidden()
    hidden_5 = decoder.init_hidden()
    hidden_6 = decoder.init_hidden()
    hidden_7 = decoder.init_hidden()
    cell_1 = decoder.init_cell()
    cell_2 = decoder.init_cell()
    cell_3 = decoder.init_cell()
    cell_4 = decoder.init_cell()
    cell_5 = decoder.init_cell()
    cell_6 = decoder.init_cell()
    cell_7 = decoder.init_cell()
    
    loss = 0
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    temp = []
    temp_score = []
    
    decoder_input = input_tensor[0]
    
    for di in range(0, target_length):
        decoder_output, (hidden_1, cell_1), (hidden_2, cell_2), (hidden_3, cell_3), (hidden_4, cell_4), (hidden_5, cell_5), (hidden_6, cell_6), (hidden_7, cell_7) = decoder(decoder_input, 
                        (hidden_1, cell_1), (hidden_2, cell_2), (hidden_3, cell_3), (hidden_4, cell_4), (hidden_5, cell_5), (hidden_6, cell_6), (hidden_7, cell_7))
        if verbose:
            temp.append(int(torch.argmax(decoder_output, dim = 1).cpu().numpy()))
            temp_score.append(decoder_output)

        #print(decoder_output.shape)
        #print(target_tensor[di].unsqueeze(0).shape)
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0).long())
        #loss += focal_loss(2, 5, criterion,decoder_output, target_tensor[di].squeeze())


        if verbose:
            print(decoder_output, target_tensor[di])
            #print(focal_loss(2, 5, criterion,decoder_output, target_tensor[di].squeeze()))
        #loss += criterion(decoder_output, target_tensor[di].long())
        if di + 1 < target_length:
            decoder_input = input_tensor[di + 1]

    loss.backward()

    if verbose:
        print("Prediction :", temp) 
        print("Score :", temp_score)
        print("Target:", target_tensor.squeeze())
    

    decoder_optimizer.step()

    return loss.item() / target_length

In [10]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [11]:
def trainIters(decoder, n_iters, print_every = 1000, plot_every = 100, learning_rate = 0.01, CEL_weight=[1,5], total_batch = maximum_target):    
    start = time.time()
    
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    decoder_optimizer = optim.Adagrad(decoder.parameters(), lr = learning_rate)
    
    criterion = nn.CrossEntropyLoss(weight = torch.Tensor(CEL_weight).to(device))
    
    scheduler = optim.lr_scheduler.StepLR(decoder_optimizer, step_size = 100, gamma = 0.5)
    
    
    for iter in range(1, n_iters + 1):
        scheduler.step()
        num = iter % total_batch
        verbose = (iter % print_every == 0)
        input_tensor = train_X[num].to(device)
        target_tensor = target_Tensor[num].to(device)
        input_tensor = Variable(input_tensor, requires_grad = True)
        #print(input_tensor.shape, target_tensor.shape)
        if input_tensor.shape[0] != target_tensor.shape[0]:
            continue
        
        loss = train(input_tensor, target_tensor, decoder, 
                     decoder_optimizer, criterion, verbose = verbose)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


In [86]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


input_size = 2
hidden_size = 256
output_size = 2

decoder = DecoderRNN(input_size, hidden_size, output_size).to(device)

trainIters(decoder, 10000, print_every=100, learning_rate=1e-2, total_batch=10)
    

tensor([[ 0.9855,  0.0145]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9970,  0.0030]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9944,  0.0056]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9790,  0.0210]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.8835,  0.1165]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.5880,  0.4120]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.3955,  0.6045]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3500,  0.6500]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3417,  0.6583]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3403,  0.6597]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3402,  0.6598]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3402,  0.6598]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3402,  0.6598]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.3400,  0.6600]], device='cuda:0') tenso

tensor([[ 0.9971,  0.0029]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9998,  0.0002]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9999,  0.0001]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9996,  0.0004]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9971,  0.0029]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9397,  0.0603]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.1969,  0.8031]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0544,  0.9456]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0539,  0.9461]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0748,  0.9252]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.1459,  0.8541]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.3558,  0.6442]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.6726,  0.3274]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.8519,  0.1481]], device='cuda:0') tenso

tensor([[ 0.9979,  0.0021]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9999,  0.0001]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9999,  0.0001]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9998,  0.0002]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9979,  0.0021]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9432,  0.0568]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.1368,  0.8632]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0218,  0.9782]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0177,  0.9823]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0233,  0.9767]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.0516,  0.9484]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.1975,  0.8025]], device='cuda:0') tensor(1, device='cuda:0')
tensor([[ 0.6403,  0.3597]], device='cuda:0') tensor(0, device='cuda:0')
tensor([[ 0.9182,  0.0818]], device='cuda:0') tenso

KeyboardInterrupt: 

In [13]:
for i in range(100000):
    num = i % maximum_target
    input_tensor = train_X[num].to(device)
    target_tensor = target_Tensor[num].to(device)
    input_tensor = Variable(input_tensor, requires_grad = True)
    print(num, input_tensor.shape, target_tensor.shape)
    if input_tensor.shape[0]<2:
        continue
    if input_tensor.shape[0] != target_tensor.shape[0]:
        continue
    

0 torch.Size([35, 2]) torch.Size([35])
1 torch.Size([22, 2]) torch.Size([22])
2 torch.Size([30, 2]) torch.Size([30])
3 torch.Size([27, 2]) torch.Size([27])
4 torch.Size([34, 2]) torch.Size([34])
5 torch.Size([24, 2]) torch.Size([24])
6 torch.Size([32, 2]) torch.Size([32])
7 torch.Size([27, 2]) torch.Size([27])
8 torch.Size([34, 2]) torch.Size([34])
9 torch.Size([37, 2]) torch.Size([37])
10 torch.Size([40, 2]) torch.Size([40])
11 torch.Size([35, 2]) torch.Size([35])
12 torch.Size([52, 2]) torch.Size([52])
13 torch.Size([36, 2]) torch.Size([36])
14 torch.Size([41, 2]) torch.Size([41])
15 torch.Size([27, 2]) torch.Size([27])
16 torch.Size([19, 2]) torch.Size([19])
17 torch.Size([24, 2]) torch.Size([24])
18 torch.Size([28, 2]) torch.Size([28])
19 torch.Size([21, 2]) torch.Size([21])
20 torch.Size([30, 2]) torch.Size([30])
21 torch.Size([22, 2]) torch.Size([22])
22 torch.Size([32, 2]) torch.Size([32])
23 torch.Size([26, 2]) torch.Size([26])
24 torch.Size([23, 2]) torch.Size([23])
25 torch.S

242 torch.Size([12, 2]) torch.Size([12])
243 torch.Size([21, 2]) torch.Size([21])
244 torch.Size([23, 2]) torch.Size([23])
245 torch.Size([15, 2]) torch.Size([15])
246 torch.Size([22, 2]) torch.Size([22])
247 torch.Size([19, 2]) torch.Size([19])
248 torch.Size([6, 2]) torch.Size([6])
249 torch.Size([31, 2]) torch.Size([31])
250 torch.Size([24, 2]) torch.Size([24])
251 torch.Size([33, 2]) torch.Size([33])
252 torch.Size([24, 2]) torch.Size([24])
253 torch.Size([24, 2]) torch.Size([24])
254 torch.Size([24, 2]) torch.Size([24])
255 torch.Size([30, 2]) torch.Size([30])
256 torch.Size([22, 2]) torch.Size([22])
257 torch.Size([35, 2]) torch.Size([35])
258 torch.Size([24, 2]) torch.Size([24])
259 torch.Size([27, 2]) torch.Size([27])
260 torch.Size([24, 2]) torch.Size([24])
261 torch.Size([17, 2]) torch.Size([17])
262 torch.Size([31, 2]) torch.Size([31])
263 torch.Size([30, 2]) torch.Size([30])
264 torch.Size([32, 2]) torch.Size([32])
265 torch.Size([18, 2]) torch.Size([18])
266 torch.Size([31

519 torch.Size([33, 2]) torch.Size([33])
520 torch.Size([26, 2]) torch.Size([26])
521 torch.Size([26, 2]) torch.Size([26])
522 torch.Size([30, 2]) torch.Size([30])
523 torch.Size([27, 2]) torch.Size([27])
524 torch.Size([31, 2]) torch.Size([31])
525 torch.Size([33, 2]) torch.Size([33])
526 torch.Size([32, 2]) torch.Size([32])
527 torch.Size([21, 2]) torch.Size([21])
528 torch.Size([34, 2]) torch.Size([34])
529 torch.Size([23, 2]) torch.Size([23])
530 torch.Size([20, 2]) torch.Size([20])
531 torch.Size([25, 2]) torch.Size([25])
532 torch.Size([32, 2]) torch.Size([32])
533 torch.Size([27, 2]) torch.Size([27])
534 torch.Size([22, 2]) torch.Size([22])
535 torch.Size([21, 2]) torch.Size([21])
536 torch.Size([23, 2]) torch.Size([23])
537 torch.Size([15, 2]) torch.Size([15])
538 torch.Size([16, 2]) torch.Size([16])
539 torch.Size([16, 2]) torch.Size([16])
540 torch.Size([16, 2]) torch.Size([16])
541 torch.Size([16, 2]) torch.Size([16])
542 torch.Size([19, 2]) torch.Size([19])
543 torch.Size([

795 torch.Size([14, 2]) torch.Size([14])
796 torch.Size([16, 2]) torch.Size([16])
797 torch.Size([24, 2]) torch.Size([24])
798 torch.Size([30, 2]) torch.Size([30])
799 torch.Size([19, 2]) torch.Size([19])
800 torch.Size([28, 2]) torch.Size([28])
801 torch.Size([20, 2]) torch.Size([20])
802 torch.Size([14, 2]) torch.Size([14])
803 torch.Size([29, 2]) torch.Size([29])
804 torch.Size([27, 2]) torch.Size([27])
805 torch.Size([19, 2]) torch.Size([19])
806 torch.Size([25, 2]) torch.Size([25])
807 torch.Size([22, 2]) torch.Size([22])
808 torch.Size([24, 2]) torch.Size([24])
809 torch.Size([27, 2]) torch.Size([27])
810 torch.Size([21, 2]) torch.Size([21])
811 torch.Size([6, 2]) torch.Size([6])
812 torch.Size([29, 2]) torch.Size([29])
813 torch.Size([37, 2]) torch.Size([37])
814 torch.Size([43, 2]) torch.Size([43])
815 torch.Size([38, 2]) torch.Size([38])
816 torch.Size([30, 2]) torch.Size([30])
817 torch.Size([42, 2]) torch.Size([42])
818 torch.Size([35, 2]) torch.Size([35])
819 torch.Size([36

1073 torch.Size([32, 2]) torch.Size([32])
1074 torch.Size([33, 2]) torch.Size([33])
1075 torch.Size([29, 2]) torch.Size([29])
1076 torch.Size([28, 2]) torch.Size([28])
1077 torch.Size([2, 2]) torch.Size([2])
1078 torch.Size([26, 2]) torch.Size([26])
1079 torch.Size([26, 2]) torch.Size([26])
1080 torch.Size([29, 2]) torch.Size([29])
1081 torch.Size([25, 2]) torch.Size([25])
1082 torch.Size([32, 2]) torch.Size([32])
1083 torch.Size([29, 2]) torch.Size([29])
1084 torch.Size([22, 2]) torch.Size([22])
1085 torch.Size([29, 2]) torch.Size([29])
1086 torch.Size([26, 2]) torch.Size([26])
1087 torch.Size([28, 2]) torch.Size([28])
1088 torch.Size([26, 2]) torch.Size([26])
1089 torch.Size([31, 2]) torch.Size([31])
1090 torch.Size([30, 2]) torch.Size([30])
1091 torch.Size([27, 2]) torch.Size([27])
1092 torch.Size([30, 2]) torch.Size([30])
1093 torch.Size([25, 2]) torch.Size([25])
1094 torch.Size([25, 2]) torch.Size([25])
1095 torch.Size([26, 2]) torch.Size([26])
1096 torch.Size([20, 2]) torch.Size(

...

17646 torch.Size([27, 2]) torch.Size([27])
17647 torch.Size([29, 2]) torch.Size([29])
17648 torch.Size([31, 2]) torch.Size([31])
17649 torch.Size([31, 2]) torch.Size([31])
17650 torch.Size([27, 2]) torch.Size([27])
17651 torch.Size([29, 2]) torch.Size([29])
17652 torch.Size([27, 2]) torch.Size([27])
17653 torch.Size([25, 2]) torch.Size([25])
17654 torch.Size([26, 2]) torch.Size([26])
17655 torch.Size([27, 2]) torch.Size([27])
17656 torch.Size([31, 2]) torch.Size([31])
17657 torch.Size([30, 2]) torch.Size([30])
17658 torch.Size([39, 2]) torch.Size([39])
17659 torch.Size([26, 2]) torch.Size([26])
17660 torch.Size([37, 2]) torch.Size([37])
17661 torch.Size([32, 2]) torch.Size([32])
17662 torch.Size([30, 2]) torch.Size([30])
17663 torch.Size([35, 2]) torch.Size([35])
17664 torch.Size([39, 2]) torch.Size([39])
17665 torch.Size([38, 2]) torch.Size([38])
17666 torch.Size([24, 2]) torch.Size([24])
17667 torch.Size([22, 2]) torch.Size([22])
17668 torch.Size([28, 2]) torch.Size([28])
17669 torch

17983 torch.Size([24, 2]) torch.Size([24])
17984 torch.Size([17, 2]) torch.Size([17])
17985 torch.Size([20, 2]) torch.Size([20])
17986 torch.Size([23, 2]) torch.Size([23])
17987 torch.Size([13, 2]) torch.Size([13])
17988 torch.Size([28, 2]) torch.Size([28])
17989 torch.Size([29, 2]) torch.Size([29])
17990 torch.Size([25, 2]) torch.Size([25])
17991 torch.Size([23, 2]) torch.Size([23])
17992 torch.Size([26, 2]) torch.Size([26])
17993 torch.Size([21, 2]) torch.Size([21])
17994 torch.Size([23, 2]) torch.Size([23])
17995 torch.Size([22, 2]) torch.Size([22])
17996 torch.Size([30, 2]) torch.Size([30])
17997 torch.Size([23, 2]) torch.Size([23])
17998 torch.Size([20, 2]) torch.Size([20])
17999 torch.Size([36, 2]) torch.Size([36])
18000 torch.Size([20, 2]) torch.Size([20])
18001 torch.Size([20, 2]) torch.Size([20])
18002 torch.Size([23, 2]) torch.Size([23])
18003 torch.Size([23, 2]) torch.Size([23])
18004 torch.Size([19, 2]) torch.Size([19])
18005 torch.Size([22, 2]) torch.Size([22])
18006 torch

18306 torch.Size([24, 2]) torch.Size([24])
18307 torch.Size([30, 2]) torch.Size([30])
18308 torch.Size([21, 2]) torch.Size([21])
18309 torch.Size([24, 2]) torch.Size([24])
18310 torch.Size([23, 2]) torch.Size([23])
18311 torch.Size([28, 2]) torch.Size([28])
18312 torch.Size([24, 2]) torch.Size([24])
18313 torch.Size([26, 2]) torch.Size([26])
18314 torch.Size([29, 2]) torch.Size([29])
18315 torch.Size([25, 2]) torch.Size([25])
18316 torch.Size([29, 2]) torch.Size([29])
18317 torch.Size([30, 2]) torch.Size([30])
18318 torch.Size([27, 2]) torch.Size([27])
18319 torch.Size([26, 2]) torch.Size([26])
18320 torch.Size([24, 2]) torch.Size([24])
18321 torch.Size([24, 2]) torch.Size([24])
18322 torch.Size([27, 2]) torch.Size([27])
18323 torch.Size([28, 2]) torch.Size([28])
18324 torch.Size([27, 2]) torch.Size([27])
18325 torch.Size([24, 2]) torch.Size([24])
18326 torch.Size([24, 2]) torch.Size([24])
18327 torch.Size([24, 2]) torch.Size([24])
18328 torch.Size([7, 2]) torch.Size([7])
18329 torch.S

18544 torch.Size([21, 2]) torch.Size([21])
18545 torch.Size([18, 2]) torch.Size([18])
18546 torch.Size([18, 2]) torch.Size([18])
18547 torch.Size([7, 2]) torch.Size([7])
18548 torch.Size([22, 2]) torch.Size([22])
18549 torch.Size([21, 2]) torch.Size([21])
18550 torch.Size([21, 2]) torch.Size([21])
18551 torch.Size([19, 2]) torch.Size([19])
18552 torch.Size([14, 2]) torch.Size([14])
18553 torch.Size([20, 2]) torch.Size([20])
18554 torch.Size([28, 2]) torch.Size([28])
18555 torch.Size([22, 2]) torch.Size([22])
18556 torch.Size([21, 2]) torch.Size([21])
18557 torch.Size([21, 2]) torch.Size([21])
18558 torch.Size([19, 2]) torch.Size([19])
18559 torch.Size([14, 2]) torch.Size([14])
18560 torch.Size([20, 2]) torch.Size([20])
18561 torch.Size([28, 2]) torch.Size([28])
18562 torch.Size([21, 2]) torch.Size([21])
18563 torch.Size([16, 2]) torch.Size([16])
18564 torch.Size([16, 2]) torch.Size([16])
18565 torch.Size([22, 2]) torch.Size([22])
18566 torch.Size([28, 2]) torch.Size([28])
18567 torch.S

KeyboardInterrupt: 

In [68]:
def evaluate(decoder, test_X):
    input_length = input_tensor.size(0)
    
    loss = 0
        
    hidden_1 = decoder.init_hidden()
    hidden_2 = decoder.init_hidden()
    hidden_3 = decoder.init_hidden()
    hidden_4 = decoder.init_hidden()
    hidden_5 = decoder.init_hidden()
    hidden_6 = decoder.init_hidden()
    hidden_7 = decoder.init_hidden()
    cell_1 = decoder.init_cell()
    cell_2 = decoder.init_cell()
    cell_3 = decoder.init_cell()
    cell_4 = decoder.init_cell()
    cell_5 = decoder.init_cell()
    cell_6 = decoder.init_cell()
    cell_7 = decoder.init_cell()
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    temp = []
    temp_score = []
    
    decoder_input = input_tensor[0]
    
    for di in range(0, input_length):
        decoder_output, (hidden_1, cell_1), (hidden_2, cell_2), (hidden_3, cell_3),  (hidden_4, cell_4), (hidden_5, cell_5), (hidden_6, cell_6), (hidden_7, cell_7) = decoder(decoder_input, 
                        (hidden_1, cell_1), (hidden_2, cell_2), (hidden_3, cell_3), (hidden_4, cell_4), (hidden_5, cell_5), (hidden_6, cell_6), (hidden_7, cell_7))
        temp.append(int(torch.argmax(decoder_output, dim = 1).cpu().numpy()))
        temp_score.append(decoder_output)
        if di + 1 < input_length:
            decoder_input = input_tensor[di+1]

    return temp

In [71]:
for i in range(5):
    target_tensor = target_Tensor[i-1].to(device)
    input_tensor = input_transform(train_X, i-1).to(device)
    print(evaluate(decoder, input_tensor))
    print(target_tensor.squeeze())

[1, 0, 0, 0, 0, 0, 1]
tensor([ 1.,  0.,  0.,  0.,  0.,  0.,  1.], device='cuda:0')
[0, 0, 0, 0, 0, 0, 0, 0]
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], device='cuda:0')
[0, 0, 0, 1, 0, 0, 0, 0]
tensor([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.], device='cuda:0')
[0, 0, 0, 0, 0, 0, 0, 1]
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.], device='cuda:0')
[0, 0, 0, 0, 0, 0, 0, 0]
tensor([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], device='cuda:0')
